In [1]:
import numpy as np
import skimage
import utils
import pathlib

In [36]:
def otsu_thresholding(im: np.ndarray) -> int:
    """
        Otsu's thresholding algorithm that segments an image into 1 or 0 (True or False)
        The function takes in a grayscale image and outputs a boolean image

        args:
            im: np.ndarray of shape (H, W) in the range [0, 255] (dtype=np.uint8)
        return:
            (int) the computed thresholding value
    """
    assert im.dtype == np.uint8
    ### START YOUR CODE HERE ### (You can change anything inside this block) 
    # You can also define other helper functions
    # Compute normalized histogram
    L = 256
    M = im.shape[0]
    N = im.shape[1]

    hist = np.zeros(L)

    unique, counts = np.unique(im, return_counts=True)

    for i, k in enumerate(unique):
        # Fill values in the histogram
        hist[k] = counts[i]

    # Normalize the histogram
    hist = hist / (M*N)

    assert np.sum(hist) == 1 # Verify that it sums to 1

    P1 = np.cumsum(hist)
    P2 = 1- P1

    cumulative_mean = np.cumsum(np.arange(L) * hist)

    global_mean = cumulative_mean[-1]
    sigma_b = (global_mean * P1 - cumulative_mean)**2 / (P1 * P2)
    sigma_b = np.nan_to_num(sigma_b) # Fill NaN values that appears because P1*P2 contains 0

    assert len(sigma_b) == L

    threshold = np.argmax(sigma_b)


    return threshold
    ### END YOUR CODE HERE ###

In [37]:
if __name__ == "__main__":
    # DO NOT CHANGE
    impaths_to_segment = [
        pathlib.Path("thumbprint.png"),
        pathlib.Path("polymercell.png")
    ]
    for impath in impaths_to_segment:
        im = utils.read_image(impath)
        threshold = otsu_thresholding(im)
        print("Found optimal threshold:", threshold)

        # Segment the image by threshold
        segmented_image = (im >= threshold)
        assert im.shape == segmented_image.shape, "Expected image shape ({}) to be same as thresholded image shape ({})".format(
                im.shape, segmented_image.shape)
        assert segmented_image.dtype == np.bool, "Expected thresholded image dtype to be np.bool. Was: {}".format(
                segmented_image.dtype)

        segmented_image = utils.to_uint8(segmented_image)

        save_path = "{}-segmented.png".format(impath.stem)
        utils.save_im(save_path, segmented_image)

Reading image: images\thumbprint.png
Found optimal threshold: 153
Saving image to: image_processed\thumbprint-segmented.png


C:\Users\joste\AppData\Local\Temp/ipykernel_28988/4185752406.py:36: RuntimeWarning: invalid value encountered in true_divide
  sigma_b = (global_mean * P1 - cumulative_mean)**2 / (P1 * P2)
C:\Users\joste\AppData\Local\Temp/ipykernel_28988/1997791817.py:16: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  assert segmented_image.dtype == np.bool, "Expected thresholded image dtype to be np.bool. Was: {}".format(


Reading image: images\polymercell.png
Found optimal threshold: 181
Saving image to: image_processed\polymercell-segmented.png
